# 使用 qianfan sdk 构建本地自动评估模型



千帆大模型平台提供了在线进行自动评估的方式，而有时我们希望有更灵活的自动评估方式。

本文将以评估文本摘要效果为例子来介绍如何使用千帆 sdk 自定义自动评估模型。

此处使用0.2.7版本以上的qianfan sdk。

In [1]:
# !pip install -U "qianfan>=0.2.7"

进行鉴权认证。

In [13]:
import os

os.environ['QIANFAN_ACCESS_KEY'] = 'your_access_key'
os.environ['QIANFAN_SECRET_KEY'] = 'your_secret_key'

# 进行评估时，请按实际情况填写 QPS LIMIT, 否则会报错
os.environ["QIANFAN_QPS_LIMIT"] = "1"
os.environ['QIANFAN_LLM_API_RETRY_COUNT'] = "3"

首先构造待评估数据集，此处用本地数据集，也可以拉取千帆平台上的数据集

In [3]:
from qianfan.dataset import Dataset

ds = Dataset.load(data_file="data_summerize/excerpt.jsonl", organize_data_as_group=True)

print(ds[0])

{'prompt': '新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。', 'response': [['修改后的立法法全文公布']], '_group': 0}


然后创建prompt模版，用于引导模型进行评估。

由于摘要任务没有标准答案，所以此处模板不提供，如果是评估问答任务，则需要提供标准答案以评估正确率。

In [4]:
evaluation_prompt_template = """
你是一名裁判员，负责为给定新闻的摘要评分。

评价标准：

{criteria}

请你遵照以下的评分步骤：
{steps}


例子：

新闻：

{prompt}

摘要：

{response}


根据答案的综合水平给出0到{max_score}之间的整数评分。
如果答案存在明显的不合理之处，则应给出一个较低的评分。
如果答案符合以上要求并且与参考答案含义相似，则应给出一个较高的评分

你的回答模版如下(只输出整数评分)：

{metric_name}
"""

接着指定评估指标和评估步骤。

此处从四个维度进行评估，分别是相关性、连贯性、一致性、流畅度

相关性

In [5]:
relevance_metric = """
相关性 - 摘要中重要内容的选择。\ 
摘要只应包含来自源文档的重要信息。 \
惩罚包含冗余和多余信息的摘要。
"""

relevance_steps = """
1. 仔细阅读摘要和源文档。
2. 将摘要与源文档进行比较，并识别文章的主要观点。
3. 评估摘要是否涵盖了文章的主要观点，以及它包含多少无关或冗余信息。
"""

relevance_max_score = 10

连贯性

In [6]:

coherence_metric = """
连贯性 - 所有句子的总体质量。 \
我们将此维度与 DUC 质量问题结构性和连贯性相关， \
其中“摘要应具有良好的结构和组织，不应只是相关信息的堆叠，而应基于从句子到主题有关的信息的连贯性主体进行构建。” \

"""

coherence_steps = """
1. 仔细阅读文章并识别主要主题和关键点。
2. 阅读摘要并与文章进行比较。检查摘要是否涵盖了文章的主要主题和关键点，并以清晰且逻辑的顺序呈现它们
"""

coherence_max_score = 10

一致性

In [7]:
consistency_metric = """
一致性 - 摘要与摘要源的客观对齐。 \
客观一致的摘要只包含与源文档支持的陈述。 \
惩罚包含幻觉事实的摘要。
"""

consistency_steps = """
1. 仔细阅读文章并识别主要事实和细节。
2. 阅读摘要并与文章进行比较。检查摘要是否包含任何不支持的文章的事实错误。
3. 基于评估标准，为一致性分配分数。
"""

consistency_max_score = 10

流畅度

In [8]:
fluency_metric = """
流畅度：摘要的语法、拼写、标点、单词选择和句子结构的质量。
1：差。摘要有很多错误，使它难以理解或听起来不自然。
2：中。摘要有一些影响文本清晰度或流畅性的错误，但要点仍然可理解。
3：好。摘要很少或没有错误，易于阅读和遵循。
"""

fluency_steps = """
读取摘要并基于给定的标准评估其流畅度。从 1 到 3 分配一个流畅度分数。
"""

fluency_max_score = 3

用ChatCompletion模型构造评估函数

In [9]:
import qianfan
import time

def get_geval_score(chat_comp,**kwargs):
    prompt = evaluation_prompt_template.format(**kwargs)
    msg = qianfan.Messages()
    msg.append(prompt, role='user')
    resp = chat_comp.do(
        messages=msg,
        temperature=0.1,
        top_p=1,
    )
    return resp['result']

最后遍历数据集进行评估

In [10]:
evaluation_metrics = {
    "Relevance": (relevance_metric, relevance_metric, relevance_max_score),
    "Coherence": (coherence_metric, coherence_steps, coherence_max_score),
    "Consistency": (consistency_metric, consistency_steps, consistency_max_score),
    "Fluency": (fluency_metric, fluency_steps, fluency_max_score),
}

In [11]:
chat_comp = qianfan.ChatCompletion(model="ERNIE-Bot-4")
result = {"Evaluation Type": [], "News id": [], "Score": []}
for eval_type, (criteria, steps, max_score) in evaluation_metrics.items():
    for ind, data in enumerate(ds):
        result["Evaluation Type"].append(eval_type)
        result["News id"].append(ind)
        evaluate = get_geval_score(
            chat_comp,
            criteria=criteria,
            steps=steps,
            prompt=data['prompt'],
            response=data['response'],
            max_score=max_score,
            metric_name=eval_type      
        )
        score = int(evaluate.strip())
        result["Score"].append(evaluate)


查看最终结果

In [12]:
import pandas as pd
pivot_df = pd.DataFrame(result, index=None).pivot(
    columns="Evaluation Type", index="News id", values="Score"
).astype(int)
pivot_df['Mean_Score'] = pivot_df.mean(axis=1)
display(pivot_df)

Evaluation Type,Coherence,Consistency,Fluency,Relevance,Mean_Score
News id,,,,,
0,8,8,2,8,6.5
1,8,8,2,8,6.5
2,5,5,2,6,4.5
